<a href="https://colab.research.google.com/github/zhangxs131/Spider/blob/main/requests_lxml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#requests+lxml 爬取voanews

*       搜索词为querys数组
*       time可设置month，day，year，表示过去一天，一个月，一年的新闻
*       pp为每页显示几个
*       pages 为最多爬取多少页
*       start_time 为具体新闻开始时间设置

这里使用线程池加快爬取，我这里写的多线程有些问题，待修改


In [ ]:
import requests
import lxml.etree as etree
import datetime
import json
import os
from concurrent.futures import ThreadPoolExecutor
import threading
from threading import Lock

pool = ThreadPoolExecutor(max_workers=10)



querys=['Ukraine','Russian']
time='month'
pp='10'
pages=30
start_time='2022-02-16 00:00:00'


ori_url='https://www.voanews.com/s?k={}&tab=all&pi={}&r={}&pp={}'

result={}
sum=0

def thread_action(li):
  sample={}
  try:
    sample['title']=li.xpath('.//h4[@class="media-block__title media-block__title--size-3"]/text()')[0].strip('\n')
    link=j.xpath('.//div[@class="media-block__content media-block__content--h media-block__content--h-xs"]/a/@href')
    #获取文章页面
    page_url='https://www.voanews.com'+link[0]
    page_res=requests.get(page_url)
    t=etree.HTML(page_res.text)
    sample['text']=t.xpath('//div[@class="wsw"]/p/text()')

    if len(sample['text'])==0:
      return {}
    print('title: ',sample['title'])
    this_time=t.xpath('//span[@class="date"]/time/@datetime')
    sample['time']=('-').join(this_time[0].split('-')[:-1]).replace('T',' ')
    # sample['time']= datetime.datetime.strptime(('-').join(this_time[0].split('-')[:-1]),'%Y-%m-%dT%H:%M:%S')

    return sample
  except:
    return {}

for query in querys:
  print('\n\n____________________________under searching keywork {}_________________________________'.format(query))

  data=[]
  for i in range(pages):
    url='https://www.voanews.com/s?k={}&tab=all&pi={}&r={}&pp={}'.format(query,i+1,time,pp)
    res=requests.get(url)

    #防止超出序列
    if url!=res.url:
      break
    lis = etree.HTML(res.text).xpath('//div[@class="media-block-wrap"]/div/ul/li')

    for j in lis:
      #多线程
      future= pool.submit(thread_action, j)

      if future.result()=={}:
        continue
      sum+=1
      if sum%50==0:
        print('___________________________Sum: {} News Time: {} ______________________'.format(sum,future.result()['time']))

      data.append(future.result())

    if future.result()!={} and future.result()['time']<start_time:
      break


  result[query]=data
  data=[]

pool.shutdown()

#save data
with open('voanews.json'.format(query),'w') as f:
  f.write(json.dumps(result))

    
